In [1804]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import ast
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, hstack

project_dir = Path("C:/Users/adbou/source/repos/KFHXRelatedAi/")
os.chdir(project_dir)

from Configs.GeneralPaths import SOURCEDATA

In [1805]:
user_transactions = pd.read_excel(Path(SOURCEDATA / "Transaction_User.xlsx"))
new_user_transaction = user_transactions.drop(columns=['TrxId'])

deals_data = pd.read_excel(Path(SOURCEDATA / "Cleaned_Deals.xlsx"))
deals_data = deals_data.drop(columns=['Unnamed: 0'])

deals_embeddings = pd.read_csv(Path(SOURCEDATA / "Deals_Embeddings.csv"))
deals_embeddings['ada_embedding'] = deals_embeddings['ada_embedding'].apply(ast.literal_eval)

new_user_transaction = new_user_transaction.merge(deals_data[['ContentId', 'Categories','Deal Type']], left_on='FK_ContentId', right_on='ContentId', how='left')
new_user_transaction = new_user_transaction.drop(columns=['ContentId'])

In [1806]:
new_user_transaction["Categories"] = new_user_transaction["Categories"].apply(lambda x: "Food and Beverage" if x == "F&B" else x)

In [1807]:
new_user_transaction.head()

,FK_BusinessUserId,PointsRedeemed,FK_ContentId,Categories,Deal Type
0,976480,1000,113923,Health & Beauty,Discount
1,976480,10000,113853,Retail,Discount
2,976480,10000,113853,Retail,Discount
3,976921,10,113851,Food and Beverage,Subscription
4,976480,50,113835,Travel,Voucher


In [1808]:
new_user_transaction['LogPointsRedeemed'] = np.log1p(new_user_transaction['PointsRedeemed'])

In [1809]:
user_item_matrix = new_user_transaction.pivot_table(index='FK_BusinessUserId', columns='FK_ContentId', values='PointsRedeemed',aggfunc="count",fill_value=0)


In [1810]:
user_item_matrix.head()

FK_ContentId,113816,113817,113819,113823,113824,113829,113830,113833,113834,113835,...,115217,115218,115221,115223,115225,115227,115229,115231,115244,115259
FK_BusinessUserId,,,,,,,,,,,,,,,,,,,,,
976480,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
976481,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
976482,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
976484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
976485,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1811]:
sparse_user_item = csr_matrix(user_item_matrix)


In [1812]:
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares(factors=30, regularization=0.3, iterations=80)

In [1813]:
alpha_val = 10
data_conf = (sparse_user_item * alpha_val).astype('double')
model.fit(data_conf)

100%|██████████| 80/80 [00:03<00:00, 23.58it/s]


In [1814]:

def recommend_items(user_id, user_item_matrix, model, deal_embeddings, n_similar_items=10):
    user_index = list(user_item_matrix.index).index(user_id)
    user_interactions = sparse_user_item[user_index]
    
    
    interacted_items_indices = user_interactions.indices
    
    
    item_factors = model.item_factors
    
    
    filtered_embeddings = deal_embeddings[deal_embeddings['ContentId'].isin(user_item_matrix.columns)]
    
    
    deal_embeddings_array = np.array(filtered_embeddings['ada_embedding'].tolist())
    
    
    similarity_matrix_factors = cosine_similarity(item_factors)
    
    
    similarity_matrix_embeddings = cosine_similarity(deal_embeddings_array)
    
    
    min_shape = min(similarity_matrix_factors.shape[0], similarity_matrix_embeddings.shape[0])
    similarity_matrix_factors = similarity_matrix_factors[:min_shape, :min_shape]
    similarity_matrix_embeddings = similarity_matrix_embeddings[:min_shape, :min_shape]
    
    
    combined_similarity_matrix = (similarity_matrix_factors + similarity_matrix_embeddings) / 2.0
    
    unique_similar_items = set()
    similar_items_with_scores = []
    
    for item_index in interacted_items_indices:
        similar_items = combined_similarity_matrix[item_index].argsort()[::-1][1:n_similar_items+1]
        for similar_item in similar_items:
            if similar_item not in unique_similar_items and similar_item not in interacted_items_indices:
                unique_similar_items.add(similar_item)
                similar_items_with_scores.append((similar_item, combined_similarity_matrix[item_index][similar_item]))
    
    
    similar_items_with_scores = sorted(similar_items_with_scores, key=lambda x: x[1], reverse=True)
    
    
    similar_item_indices = [item for item, score in similar_items_with_scores[:n_similar_items]]
    
   
    similar_item_ids = [user_item_matrix.columns[item_id] for item_id in similar_item_indices]
    similar_item_scores = [score for item, score in similar_items_with_scores[:n_similar_items]]
    
    return similar_item_ids, similar_item_scores

In [1815]:
user_id = 1064756
recommended_items = recommend_items(user_id, user_item_matrix, model,deals_embeddings)
print(f"Recommended items for user {user_id}: {recommended_items[0]}")
print(f"Scores items for user {user_id}: {recommended_items[1]}")

Recommended items for user 1064756: [115105, 113829, 115095, 115133, 113830, 115096, 113883, 113885, 113923, 113863]
Scores items for user 1064756: [0.6306608101469602, 0.5009940274761155, 0.39305333110091245, 0.37749987410292835, 0.3502361209570341, 0.3448374567005049, 0.30478214029631556, 0.27335537371375895, 0.2719053095629834, 0.2710761438760102]


In [1826]:
specific_content_id = 115095,
result = deals_data[deals_data['ContentId'] == specific_content_id]

result

,ContentId,Title,FK_StatusId,Deal Type,Description,Location,Points,Categories
140,115095,30% off from Caldo Restaurant!,2,Voucher,Enjoy 30% off from Caldo Restaurant.,https://www.google.com/maps/place/Caldo+%D9%83...,100,F&B


In [1820]:
user_last_interactions = user_item_matrix.loc[user_id]
user_interacted_items = user_last_interactions[user_last_interactions > 0].index
user_deals = deals_data[deals_data['ContentId'].isin(user_interacted_items)]
user_deals

,ContentId,Title,FK_StatusId,Deal Type,Description,Location,Points,Categories
139,115094,30% off from Car Spa!,1,Discount,Enjoy 30% off from Car Spa.,https://maps.app.goo.gl/oaPZdGGPm9yWycgR9,100,Automotive


In [1818]:
user_trs = new_user_transaction[new_user_transaction["FK_BusinessUserId"] == user_id]
user_trs

,FK_BusinessUserId,PointsRedeemed,FK_ContentId,Categories,Deal Type,LogPointsRedeemed
37103,1064756,100,115094,Automotive,Discount,4.615121


In [1819]:
# def recommend_items(user_id, user_item_matrix, model, n_similar_items=10):
#     user_index = list(user_item_matrix.index).index(user_id)
#     user_interactions = sparse_user_item[user_index]
    
#     interacted_items_indices = user_interactions.indices
#     item_factors = model.item_factors
    
#     similarity_matrix = cosine_similarity(item_factors)
    
#     unique_similar_items = set()
#     similar_items_with_scores = []
    
#     for item_index in interacted_items_indices:
#         similar_items = similarity_matrix[item_index].argsort()[::-1][1:n_similar_items+1]
#         for similar_item in similar_items:
#             if similar_item not in unique_similar_items and similar_item not in interacted_items_indices:
#                 unique_similar_items.add(similar_item)
#                 similar_items_with_scores.append((similar_item, similarity_matrix[item_index][similar_item]))
    
#     similar_items_with_scores = sorted(similar_items_with_scores, key=lambda x: x[1], reverse=True)
    
#     similar_item_indices = [item for item, score in similar_items_with_scores[:n_similar_items]]
    
#     similar_item_ids = [user_item_matrix.columns[item_id] for item_id in similar_item_indices]
#     similar_item_scores = [score for item, score in similar_items_with_scores[:n_similar_items]]
   
#     return similar_item_ids, similar_item_scores